In [1]:
import numpy as np
import pandas as pd
import os
import json
import pickle
import random
import math
import re
import tweepy
import string
from tweepy.api import TooManyRequests
import time

In [2]:
#Twitter API credentials
consumer_key = "9Wj6RDG1XbCiIUf6DOrk6siyA"
consumer_secret = "uq72hF4fCdkJVTVoUZ402KOvQhagRbrrYXHCMBR5CyWntptbPJ"
access_key = "891352855239548929-FsBhQM3mILMQXaEeT1BC03DuZbXezRV"
access_secret = "zO0WPUaoVGfMVw1t10885x6106Wuyvrnl4464ABHJotMx"

#authorize twitter, initialize tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [3]:
df_train_all = pd.read_json('raw_data/train.json')
df_test_all = pd.read_json('raw_data/test.json')
df_val_all = pd.read_json('raw_data/val.json')

In [4]:
df_train_all.columns

Index(['DTweetID', 'DguardianID', 'DguardianName', 'OriginalPosterID',
       'OriginalTweetID', 'ScreenNameOriginalPoster', 'dtweet_content',
       'label', 'otweet_content', 'url'],
      dtype='object')

In [5]:
columns = ['OriginalTweetID', 'otweet_content', 'url']

In [6]:
df_train = df_train_all[columns]
df_test = df_test_all[columns]
df_val = df_val_all[columns]

In [7]:
df_train.head()

,OriginalTweetID,otweet_content,url
0,780136889441456128,the families of these <number> should sit in t...,http://www.politifact.com/truth-o-meter/statem...
1,913075551212011520,"@user <person> didnt lower my my taxes , and h...",http://www.politifact.com/truth-o-meter/statem...
2,753045148775440384,@user like when white christian men commit ter...,http://www.politifact.com/punditfact/statement...
3,743830161393520640,@user don't believe the media ! they lie . loo...,http://www.snopes.com/donald-trumps-son-game-h...
4,818188352314900480,@user says ultra liberal <person> hating,http://www.politifact.com/truth-o-meter/statem...


In [8]:
print(df_train.shape)
print(df_test.shape)
print(df_val.shape)

(58562, 3)
(7322, 3)
(7319, 3)


In [9]:
df_train.iloc[7]

OriginalTweetID                                   931327673686564864
otweet_content     @user there's no reason <number> trump's decis...
url                http://www.snopes.com/donald-trumps-son-game-h...
Name: 7, dtype: object

In [10]:
punc = ['.',',','!','?']
def clean_up(s: str):
    if type(s) is float and math.isnan(s):
        # handling nans
        return ""
    else:
        # remove URLs
        s = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', s)
        s = re.sub(r'http\S+', '', s)
        
        # remove content in <...>
        s = re.sub(r'<[\w\d]+>', '', s)
        
        # remove special characters - ()#%^&*@! etc.
        s = ''.join(e for e in s if e.isalnum() or e is ' ')
        
        # remove multiple spaces
        s = re.sub("\s+"," ",s)
        
        return s.strip()

In [11]:
def fetch_tweet(df: pd.DataFrame):
    idxs_not_found = []
    i = 0
    while i < df.shape[0]:
        if i%100 == 0:
            print(f"At Index => {i}")
        id = df.iloc[i]['OriginalTweetID']
        try:
            status = api.get_status(id, tweet_mode="extended")
            tweet_polluted = status.full_text
            df_train.loc[i,'otweet_content'] = tweet_polluted
            i+=1
        except TooManyRequests:
            # Catches Rate Limit Exception
            print(f"Too Many Requests => {i}")
            time.sleep(60*16)  # Delay for 16 mins (1 minute buffer) as only 900 requests per 15 mins is allowed
        except Exception:
            # Catches Exception if tweet ID is not found or forbidden
            idxs_not_found.append(i)
            # tweet_polluted = df.iloc[i]['otweet_content']
            # df_train.loc[i,'otweet_content'] = clean_up(tweet_polluted)
            i+=1
        
    return df, idxs_not_found
        

In [12]:
# Creating Validation Json file
df_val_final, val_records_not_found = fetch_tweet(df_val)
df_val_final.to_json('val_tweet.json')
print(f"Out of {df_val_final.shape[0]} data samples, {val_records_not_found} tweets were not found in API")

At Index => 0
Too Many Requests => 0
At Index => 0


/home/parin/anaconda3/envs/685/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


At Index => 100
At Index => 200
At Index => 300
At Index => 400
At Index => 500
At Index => 600
At Index => 700
At Index => 800
At Index => 900
Too Many Requests => 900
At Index => 900
At Index => 1000
At Index => 1100
At Index => 1200
At Index => 1300
At Index => 1400
At Index => 1500
At Index => 1600
At Index => 1700
At Index => 1800
Too Many Requests => 1800
At Index => 1800
At Index => 1900
At Index => 2000
At Index => 2100
At Index => 2200
At Index => 2300
At Index => 2400
At Index => 2500
At Index => 2600
At Index => 2700
Too Many Requests => 2700
At Index => 2700
At Index => 2800
At Index => 2900
At Index => 3000
At Index => 3100
At Index => 3200
At Index => 3300
At Index => 3400
At Index => 3500
At Index => 3600
Too Many Requests => 3600
At Index => 3600
At Index => 3700
At Index => 3800
At Index => 3900
At Index => 4000
At Index => 4100
At Index => 4200
At Index => 4300
At Index => 4400
At Index => 4500
Too Many Requests => 4500
At Index => 4500
At Index => 4600
At Index => 47

In [13]:
# Creating Test Json file
df_test_final, test_records_not_found = fetch_tweet(df_test)
df_test_final.to_json('test_tweet.json')
print(f"Out of {df_test_final.shape[0]} data samples, {test_records_not_found} tweets were not found in API")

At Index => 0
At Index => 100
At Index => 200
At Index => 300
At Index => 400
At Index => 500
At Index => 600
At Index => 700
Too Many Requests => 781
At Index => 800
At Index => 900
At Index => 1000
At Index => 1100
At Index => 1200
At Index => 1300
At Index => 1400
At Index => 1500
At Index => 1600
Too Many Requests => 1681
At Index => 1700
At Index => 1800
At Index => 1900
At Index => 2000
At Index => 2100
At Index => 2200
At Index => 2300
At Index => 2400
At Index => 2500
Too Many Requests => 2581
At Index => 2600
At Index => 2700
At Index => 2800
At Index => 2900
At Index => 3000
At Index => 3100
At Index => 3200
At Index => 3300
At Index => 3400
Too Many Requests => 3481
At Index => 3500
At Index => 3600
At Index => 3700
At Index => 3800
At Index => 3900
At Index => 4000
At Index => 4100
At Index => 4200
At Index => 4300
Too Many Requests => 4381
At Index => 4400
At Index => 4500
At Index => 4600
At Index => 4700
At Index => 4800
At Index => 4900
At Index => 5000
At Index => 5100

In [14]:
# Creating Training Json file
df_train_final, train_records_not_found = fetch_tweet(df_train)
df_train_final.to_json('train_tweet.json')
print(f"Out of {df_train_final.shape[0]} data samples, {train_records_not_found} tweets were not found in API")

At Index => 0
At Index => 100
At Index => 200
At Index => 300
At Index => 400
At Index => 500
At Index => 600
Too Many Requests => 659
At Index => 700
At Index => 800
At Index => 900
At Index => 1000
At Index => 1100
At Index => 1200
At Index => 1300
At Index => 1400
At Index => 1500
Too Many Requests => 1559
At Index => 1600
At Index => 1700
At Index => 1800
At Index => 1900
At Index => 2000
At Index => 2100
At Index => 2200
At Index => 2300
At Index => 2400
Too Many Requests => 2459
At Index => 2500
At Index => 2600
At Index => 2700
At Index => 2800
At Index => 2900
At Index => 3000
At Index => 3100
At Index => 3200
At Index => 3300
Too Many Requests => 3359
At Index => 3400
At Index => 3500
At Index => 3600
At Index => 3700
At Index => 3800
At Index => 3900
At Index => 4000
At Index => 4100
At Index => 4200
Too Many Requests => 4260
At Index => 4300
At Index => 4400
At Index => 4500
At Index => 4600
At Index => 4700
At Index => 4800
At Index => 4900
At Index => 5000
At Index => 5100

At Index => 39700
At Index => 39800
At Index => 39900
At Index => 40000
At Index => 40100
At Index => 40200
Too Many Requests => 40260
At Index => 40300
At Index => 40400
At Index => 40500
At Index => 40600
At Index => 40700
At Index => 40800
At Index => 40900
At Index => 41000
At Index => 41100
Too Many Requests => 41160
At Index => 41200
At Index => 41300
At Index => 41400
At Index => 41500
At Index => 41600
At Index => 41700
At Index => 41800
At Index => 41900
At Index => 42000
Too Many Requests => 42060
At Index => 42100
At Index => 42200
At Index => 42300
At Index => 42400
At Index => 42500
At Index => 42600
At Index => 42700
At Index => 42800
At Index => 42900
Too Many Requests => 42960
At Index => 43000
At Index => 43100
At Index => 43200
At Index => 43300
At Index => 43400
At Index => 43500
At Index => 43600
At Index => 43700
At Index => 43800
Too Many Requests => 43860
At Index => 43900
At Index => 44000
At Index => 44100
At Index => 44200
At Index => 44300
At Index => 44400
A

In [15]:
print(f"Out of {df_val_final.shape[0]} data samples, {len(val_records_not_found)} tweets were not found in API")
print(f"Out of {df_test_final.shape[0]} data samples, {len(test_records_not_found)} tweets were not found in API")
print(f"Out of {df_train_final.shape[0]} data samples, {len(train_records_not_found)} tweets were not found in API")

Out of 7319 data samples, 4183 tweets were not found in API
Out of 7322 data samples, 4256 tweets were not found in API
Out of 58562 data samples, 33832 tweets were not found in API
